In [2]:
import numpy as np
import pandas as pd
import ccxt
from sklearn.preprocessing import StandardScaler
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.env_checker import check_env
import talib
import requests
from gym import spaces, Env

# Configuration
INITIAL_BALANCE = 10.0
TRADING_FEE = 0.10  # 10%
LOOKBACK_WINDOW = 60  # 60 periods (5 hours)
INDICATOR_WINDOW = 14
url = "https://api.binance.com/api/v3/klines"
params = {
    'symbol': 'BTCUSDT',  # The trading pair
    'interval': '1m',     # 1-minute candlesticks
    'limit': 1000        # Number of data points to fetch
}

response = requests.get(url, params=params)
data = response.json()

if response.status_code == 200:
    data = pd.DataFrame(
        data,
        columns=['timestamp', 'open', 'high', 'low', 'close', 'volume', 
                    'close_time', 'quote_asset_volume', 'number_of_trades', 
                    'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'ignore']
    )
    data = data[['timestamp', 'open', 'high', 'low', 'close', 'volume']]
    data['timestamp'] = pd.to_datetime(data['timestamp'], unit='ms')
    data.set_index('timestamp', inplace=True)
    data = data.astype(float)
else:
    print(f"Error: {response.status_code}, {response.text}")
data

,open,high,low,close,volume
timestamp,,,,,
2025-02-11 02:22:00,97660.46,97729.33,97660.46,97705.79,13.41279
2025-02-11 02:23:00,97705.78,97705.79,97629.38,97653.97,7.93944
2025-02-11 02:24:00,97653.96,97711.15,97619.43,97711.14,16.62807
2025-02-11 02:25:00,97711.15,97790.55,97711.14,97756.24,12.37841
2025-02-11 02:26:00,97756.23,97756.23,97696.98,97696.99,9.74859
...,...,...,...,...,...
2025-02-11 18:57:00,95510.38,95533.46,95509.97,95509.97,9.46631
2025-02-11 18:58:00,95509.97,95523.95,95500.27,95523.95,8.00227
2025-02-11 18:59:00,95523.94,95523.95,95423.83,95469.28,11.55498


In [3]:
import pandas as pd

# Define Indicator Functions
def SMA(data, period=14, column='close'):
    return data[column].rolling(window=period).mean()

def EMA(data, period=14, column='close'):
    return data[column].ewm(span=period, adjust=False).mean()

def ADX(data, period=14):
    high, low, close = data['high'], data['low'], data['close']
    plus_dm = high.diff().clip(lower=0)
    minus_dm = -low.diff().clip(upper=0)
    tr = pd.concat([high - low, abs(high - close.shift(1)), abs(low - close.shift(1))], axis=1).max(axis=1)
    atr = tr.rolling(window=period).mean()
    plus_di = 100 * (plus_dm.ewm(span=period, adjust=False).mean() / atr)
    minus_di = abs(100 * (minus_dm.ewm(span=period, adjust=False).mean() / atr))
    dx = (abs(plus_di - minus_di) / (plus_di + minus_di)) * 100
    return dx.rolling(window=period).mean().dropna()

def calculate_di(data, period=14):
    high = data['high']
    low = data['low']
    close = data['close']

    # Calculate the Directional Movement (DM)
    plus_dm = high.diff().clip(lower=0)
    minus_dm = -low.diff().clip(upper=0)

    # True Range (TR)
    tr = pd.concat([high - low, abs(high - close.shift(1)), abs(low - close.shift(1))], axis=1).max(axis=1)

    # Smooth the DM and TR over the period
    smoothed_plus_dm = plus_dm.rolling(window=period).sum()
    smoothed_minus_dm = minus_dm.rolling(window=period).sum()
    smoothed_tr = tr.rolling(window=period).sum()

    # Calculate +DI and -DI
    data['+DI'] = (smoothed_plus_dm / smoothed_tr) * 100
    data['-DI'] = (smoothed_minus_dm / smoothed_tr) * 100
    
    return data[['+DI', '-DI']].dropna()

def Ichimoku(data):
    data = data.copy()
    high, low = data['high'], data['low']

    data['Tenkan-sen'] = (high.rolling(window=9).max() + low.rolling(window=9).min()) / 2
    data['Kijun-sen'] = (high.rolling(window=26).max() + low.rolling(window=26).min()) / 2
    data['Senkou Span A'] = ((data['Tenkan-sen'] + data['Kijun-sen']) / 2).shift(26)
    data['Senkou Span B'] = ((high.rolling(window=52).max() + low.rolling(window=52).min()) / 2).shift(26)
    data['Chikou Span'] = data['close'].shift(-26)

    # Ensure all missing values are handled properly
    return data[['Tenkan-sen', 'Kijun-sen', 'Senkou Span A', 'Senkou Span B', 'Chikou Span']].dropna()


def RSI(data, period=14, column='close'):
    delta = data[column].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=period).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=period).mean()
    rs = gain / loss
    return (100 - (100 / (1 + rs))).dropna()

def MACD(data, short_period=12, long_period=26, signal_period=9):
    short_ema = EMA(data, short_period)
    long_ema = EMA(data, long_period)
    macd_line = short_ema - long_ema
    signal_line = macd_line.ewm(span=signal_period, adjust=False).mean()
    return macd_line.dropna(), signal_line.dropna()

def Stochastic_Oscillator(data, period=14):
    high, low, close = data['high'], data['low'], data['close']
    lowest_low = low.rolling(window=period).min()
    highest_high = high.rolling(window=period).max()
    k = 100 * ((close - lowest_low) / (highest_high - lowest_low))
    d = k.rolling(window=3).mean()
    return k.dropna(), d.dropna()

def OBV(data):
    obv = (data['close'].diff().fillna(0) > 0) * data['volume'] - (data['close'].diff().fillna(0) < 0) * data['volume']
    return obv.cumsum().dropna()

def VWAP(data):
    return ((data['close'] * data['volume']).cumsum() / data['volume'].cumsum()).dropna()

def Bollinger_Bands(data, period=20):
    sma = SMA(data, period)
    std = data['close'].rolling(window=period).std()
    upper_band = sma + (2 * std)
    lower_band = sma - (2 * std)
    return upper_band.dropna(), lower_band.dropna()

def ATR(data, period=14):
    high, low, close = data['high'], data['low'], data['close']
    tr = pd.concat([
        high - low, 
        abs(high - close.shift()), 
        abs(low - close.shift())
    ], axis=1).max(axis=1)
    return tr.rolling(window=period).mean().dropna()

def Keltner_Channel(data, period=20):
    ema = EMA(data, period)
    atr = ATR(data, period)
    upper_band = ema + (2 * atr)
    lower_band = ema - (2 * atr)
    return upper_band.dropna(), lower_band.dropna()
# Ensure 'data' is a DataFrame
if isinstance(data, list):
    data = pd.DataFrame(data)

# Calculate Indicators and Add to DataFrame
data['SMA_50'] = SMA(data, 50)
data['EMA_20'] = EMA(data, 20)
data['ADX_14'] = ADX(data, 14)
ichimoku = Ichimoku(data)
data = pd.concat([data, ichimoku], axis=1)
data[['+DI', '-DI']] = calculate_di(data, 14)
data['RSI_14'] = RSI(data, 14)
data['MACD'], data['MACD_Signal'] = MACD(data)
data['Stoch_K'], data['Stoch_D'] = Stochastic_Oscillator(data)
data['OBV'] = OBV(data)
data['VWAP'] = VWAP(data)
data['BB_Upper'], data['BB_Lower'] = Bollinger_Bands(data)
data['ATR_14'] = ATR(data, 14)
data['Keltner_Upper'], data['Keltner_Lower'] = Keltner_Channel(data)

# Drop NaN values across all calculated indicators
data = data.dropna()

# Print the last few rows of the DataFrame
print(data.tail())


                         open      high       low     close    volume  \
timestamp                                                               
2025-02-11 18:31:00  95649.49  95757.80  95649.48  95738.91  12.72812   
2025-02-11 18:32:00  95738.92  95784.58  95738.92  95784.58  13.47782   
2025-02-11 18:33:00  95784.57  95801.94  95755.15  95801.94   6.39397   
2025-02-11 18:34:00  95801.93  95833.50  95780.37  95833.50   9.65216   
2025-02-11 18:35:00  95833.50  95846.13  95775.83  95846.13  12.87486   

                         SMA_50        EMA_20     ADX_14  Tenkan-sen  \
timestamp                                                              
2025-02-11 18:31:00  95936.9362  95755.858726  52.765134   95682.900   
2025-02-11 18:32:00  95929.3720  95758.594085  52.383253   95696.290   
2025-02-11 18:33:00  95923.1378  95762.722267  52.419877   95711.825   
2025-02-11 18:34:00  95917.7674  95769.463004  53.120539   95727.605   
2025-02-11 18:35:00  95910.7702  95776.764623  51.995673

<h1>Trading Indicators Guide</h1>

<h2>1. Trend Indicators 📈 (Detect Market Direction)</h2>

<h3>(a) Simple Moving Average (SMA) & Exponential Moving Average (EMA)</h3>
<p><strong>✅ What It Detects:</strong></p>
<ul>
    <li>SMA: Long-term trend direction.</li>
    <li>EMA: Short-term trend with more weight on recent prices.</li>
</ul>
<p><strong>📊 How to Interpret Results:</strong></p>
<ul>
    <li>Price above SMA/EMA → Uptrend (Bullish signal)</li>
    <li>Price below SMA/EMA → Downtrend (Bearish signal)</li>
    <li>Golden Cross (50 EMA > 200 EMA) → Strong Bullish</li>
    <li>Death Cross (50 EMA < 200 EMA) → Strong Bearish</li>
</ul>

<h3>(b) Average Directional Index (ADX)</h3>
<p><strong>✅ What It Detects:</strong></p>
<ul>
    <li>Strength of a trend (not direction).</li>
</ul>
<p><strong>📊 How to Interpret Results:</strong></p>
<ul>
    <li>ADX > 25 → Strong trend (either bullish or bearish).</li>
    <li>ADX < 20 → Weak trend (sideways movement).</li>
    <li>Increasing ADX → Trend is gaining strength.</li>
    <li>Decreasing ADX → Trend is weakening.</li>
</ul>

<h3>(c) Ichimoku Cloud</h3>
<p><strong>✅ What It Detects:</strong></p>
<ul>
    <li>Trend direction, support/resistance, and momentum.</li>
</ul>
<p><strong>📊 How to Interpret Results:</strong></p>
<ul>
    <li>Price above the cloud → Bullish trend.</li>
    <li>Price below the cloud → Bearish trend.</li>
    <li>Price inside the cloud → Consolidation/Uncertainty.</li>
</ul>

<h2>2. Momentum Indicators 🚀 (Measure Strength of Price Movement)</h2>

<h3>(a) Relative Strength Index (RSI)</h3>
<p><strong>✅ What It Detects:</strong></p>
<ul>
    <li>Overbought and oversold conditions (potential reversal points).</li>
</ul>
<p><strong>📊 How to Interpret Results:</strong></p>
<ul>
    <li>RSI > 70 → Overbought (Sell signal).</li>
    <li>RSI < 30 → Oversold (Buy signal).</li>
    <li>Divergence (Price rises but RSI falls) → Weakening trend, possible reversal.</li>
</ul>

<h3>(b) Moving Average Convergence Divergence (MACD)</h3>
<p><strong>✅ What It Detects:</strong></p>
<ul>
    <li>Trend direction, momentum, and reversals.</li>
</ul>
<p><strong>📊 How to Interpret Results:</strong></p>
<ul>
    <li>MACD line crosses above Signal line → Bullish crossover (Buy).</li>
    <li>MACD line crosses below Signal line → Bearish crossover (Sell).</li>
    <li>MACD Divergence → Price making higher highs while MACD falls → Possible trend reversal.</li>
</ul>

<h3>(c) Stochastic Oscillator</h3>
<p><strong>✅ What It Detects:</strong></p>
<ul>
    <li>Momentum shifts and overbought/oversold conditions.</li>
</ul>
<p><strong>📊 How to Interpret Results:</strong></p>
<ul>
    <li>%K > 80 → Overbought (Sell signal).</li>
    <li>%K < 20 → Oversold (Buy signal).</li>
    <li>%K crossing %D from below → Bullish reversal.</li>
    <li>%K crossing %D from above → Bearish reversal.</li>
</ul>

<h2>3. Volume Indicators 📊 (Confirm Market Strength)</h2>

<h3>(a) On-Balance Volume (OBV)</h3>
<p><strong>✅ What It Detects:</strong></p>
<ul>
    <li>Buying vs. selling pressure.</li>
</ul>
<p><strong>📊 How to Interpret Results:</strong></p>
<ul>
    <li>Rising OBV → Buying pressure (Bullish).</li>
    <li>Falling OBV → Selling pressure (Bearish).</li>
    <li>Divergence (Price rising, OBV falling) → Weak uptrend, possible reversal.</li>
</ul>

<h3>(b) Volume Weighted Average Price (VWAP)</h3>
<p><strong>✅ What It Detects:</strong></p>
<ul>
    <li>Trend strength based on volume-weighted prices.</li>
</ul>
<p><strong>📊 How to Interpret Results:</strong></p>
<ul>
    <li>Price above VWAP → Strong bullish trend.</li>
    <li>Price below VWAP → Strong bearish trend.</li>
    <li>VWAP flat → Market is ranging (no clear trend).</li>
</ul>

<h2>4. Volatility Indicators ⚡ (Measure Market Risk & Big Moves)</h2>

<h3>(a) Bollinger Bands (BB)</h3>
<p><strong>✅ What It Detects:</strong></p>
<ul>
    <li>Volatility and potential breakouts.</li>
</ul>
<p><strong>📊 How to Interpret Results:</strong></p>
<ul>
    <li>Price touches upper band → Overbought (Sell signal).</li>
    <li>Price touches lower band → Oversold (Buy signal).</li>
    <li>Bands expand → Increased volatility (Big move expected).</li>
    <li>Bands contract → Low volatility (Possible breakout).</li>
</ul>

<h3>(b) Average True Range (ATR)</h3>
<p><strong>✅ What It Detects:</strong></p>
<ul>
    <li>Market volatility (size of price movements).</li>
</ul>
<p><strong>📊 How to Interpret Results:</strong></p>
<ul>
    <li>High ATR → High volatility (Big price swings).</li>
    <li>Low ATR → Low volatility (Stable price movement).</li>
    <li>Rising ATR → Market getting volatile (Breakout possible).</li>
</ul>

<h3>(c) Keltner Channel</h3>
<p><strong>✅ What It Detects:</strong></p>
<ul>
    <li>Trend direction and volatility.</li>
</ul>
<p><strong>📊 How to Interpret Results:</strong></p>
<ul>
    <li>Price above upper band → Strong bullish move.</li>
    <li>Price below lower band → Strong bearish move.</li>
    <li>Price inside the bands → Normal market behavior.</li>
</ul>

Strategy Type >	Best Indicator Combinations <br>
Trend-Following >	EMA + ADX + Ichimoku Cloud<br>
Momentum-Based >	RSI + MACD + Stochastic Oscillator<br>
Breakout Trading >	Bollinger Bands + ATR + OBV<br>
Mean Reversion >	RSI + Bollinger Bands + VWAP<br>
Scalping/Day Trading >	VWAP + Stochastic RSI + Volume<br>

In [18]:
def get_signal(data):
    signals = []
    indicators = ['SMA','EMA','ADX','Ichimoku','RSI','MACD','Stochastic Oscillator','OBV','VWAP','Bollinger Bands','ATR','Keltner Channel']
    # SMA
    if data['close'].iloc[-1] > data['SMA_50'].iloc[-1]:
        signals.append('bullish')
    elif data['close'].iloc[-1] < data['SMA_50'].iloc[-1]:
        signals.append('bearish')
    else:
        signals.append('neutral')

    # EMA
    if data['close'].iloc[-1] > data['EMA_20'].iloc[-1]:
        signals.append('bullish')
    elif data['close'].iloc[-1] < data['EMA_20'].iloc[-1]:
        signals.append('bearish')
    else:
        signals.append('neutral')

    # ADX
    if data['ADX_14'].iloc[-1] > 25 and data['+DI'].iloc[-1] > data['-DI'].iloc[-1]:
        signals.append('bullish')
    elif data['ADX_14'].iloc[-1] > 25 and data['+DI'].iloc[-1] < data['-DI'].iloc[-1]:
        signals.append('bearish')
    else:
        signals.append('neutral')

    # Ichimoku
    if data['close'].iloc[-1] > data['Senkou Span A'].iloc[-1] and data['Tenkan-sen'].iloc[-1] > data['Kijun-sen'].iloc[-1]:
        signals.append('bullish')
    elif data['close'].iloc[-1] < data['Senkou Span A'].iloc[-1] and data['Tenkan-sen'].iloc[-1] < data['Kijun-sen'].iloc[-1]:
        signals.append('bearish')
    else:
        signals.append('neutral')

    # RSI
    if data['RSI_14'].iloc[-1] < 30 or data['RSI_14'].iloc[-1] > 50:
        signals.append('bullish')
    elif data['RSI_14'].iloc[-1] > 30 or data['RSI_14'].iloc[-1] < 50:
        signals.append('bearish')
    else:
        signals.append('neutral')

    # MACD
    if data['MACD'].iloc[-1] > data['MACD_Signal'].iloc[-1]:
        signals.append('bullish')
    elif data['MACD'].iloc[-1] < data['MACD_Signal'].iloc[-1]:
        signals.append('bearish')
    else:
        signals.append('neutral')

    # Stochastic Oscillator
    if data['Stoch_K'].iloc[-1] > data['Stoch_D'].iloc[-1] and data['Stoch_K'].iloc[-1] < 20:
        signals.append('bullish')
    elif data['Stoch_K'].iloc[-1] < data['Stoch_D'].iloc[-1] and data['Stoch_K'].iloc[-1] > 20:
        signals.append('bearish')
    else:
        signals.append('neutral')

    # OBV
    if data['OBV'].iloc[-1] > data['OBV'].iloc[-2]:  # OBV rising
        signals.append('bullish')
    elif data['OBV'].iloc[-1] < data['OBV'].iloc[-2]:
        signals.append('bearish')
    else:
        signals.append('neutral')

    # VWAP
    if data['close'].iloc[-1] > data['VWAP'].iloc[-1]:
        signals.append('bullish')
    elif data['close'].iloc[-1] < data['VWAP'].iloc[-1]:
        signals.append('bearish')
    else:
        signals.append('neutral')

    # Bollinger Bands
    if data['close'].iloc[-1] < data['BB_Lower'].iloc[-1]:  # Touching lower band
        signals.append('bullish')
    elif data['close'].iloc[-1] > data['BB_Upper'].iloc[-1]:  # Touching upper band
        signals.append('bearish')
    else:
        signals.append('neutral')

    # ATR (Volatility Expansion)
    if data['ATR_14'].iloc[-1] > data['ATR_14'].iloc[-2]:  # ATR increasing
        signals.append('bullish')
    elif data['ATR_14'].iloc[-1] < data['ATR_14'].iloc[-2]:
        signals.append('bearish')
    else:
        signals.append('neutral')

    # Keltner Channel
    if data['close'].iloc[-1] > data['Keltner_Upper'].iloc[-1]:
        signals.append('bullish')
    elif data['close'].iloc[-1] < data['Keltner_Lower'].iloc[-1]:
        signals.append('bearish')
    else:
        signals.append('neutral')
        

    # Count bullish and bearish signals
    bullish_count = signals.count('bullish')
    bearish_count = signals.count('bearish')
    for i in range(len(indicators)):
        print(indicators[i]+' '+signals[i])
    print(bullish_count,bearish_count,len(signals)-bullish_count-bearish_count)
    # Final prediction
    if bullish_count > bearish_count:
        return 'bullish'
    elif bearish_count > bullish_count:
        return 'bearish'
    else:
        return 'neutral'  # When there's a tie or no clear trend
    

# Example: Get the final signal for the most recent data
signal = get_signal(data)
print(f"Final Prediction: {signal}")


SMA bullish
EMA bullish
ADX neutral
Ichimoku bullish
RSI bullish
MACD bullish
Stochastic Oscillator bearish
OBV bearish
VWAP bullish
Bollinger Bands neutral
ATR bearish
Keltner Channel neutral
6 3 3
Final Prediction: bullish


In [5]:
import torch
import torch.nn as nn

class TradingModel(nn.Module):
    def __init__(self, input_dim, num_layers=3, hidden_units=64):
        super().__init__()
        layers = []
        for _ in range(num_layers):
            layers.append(nn.Linear(input_dim, hidden_units))
            layers.append(nn.ReLU())
            input_dim = hidden_units
        layers.append(nn.Linear(hidden_units, 1))  # Predict return
        self.net = nn.Sequential(*layers)
    
    def forward(self, x):
        return self.net(x)

In [6]:
def maml_train(model, tasks, inner_lr, inner_steps, outer_lr, epochs):
    opt = torch.optim.Adam(model.parameters(), lr=outer_lr)
    for epoch in range(epochs):
        total_loss = 0
        for task in tasks:
            # Clone model for inner loop
            fast_weights = dict(model.named_parameters())
            
            # Inner loop adaptation
            for _ in range(inner_steps):
                loss = compute_loss(task['support'], model, fast_weights)
                grads = torch.autograd.grad(loss, fast_weights.values())
                fast_weights = {name: param - inner_lr * grad 
                               for (name, param), grad in zip(fast_weights.items(), grads)}
            
            # Compute query loss and backprop
            query_loss = compute_loss(task['query'], model, fast_weights)
            total_loss += query_loss
            query_loss.backward()
        
        # Outer optimization step
        opt.step()
        opt.zero_grad()

In [7]:
import optuna

def objective(trial):
    params = {
        'inner_lr': trial.suggest_loguniform('inner_lr', 1e-5, 1e-2),
        'outer_lr': trial.suggest_loguniform('outer_lr', 1e-4, 1e-2),
        'inner_steps': trial.suggest_int('inner_steps', 1, 5),
        'layers': trial.suggest_int('layers', 2, 4),
        'neurons': trial.suggest_categorical('neurons', [32, 64, 128])
    }
    
    model = TradingModel(input_dim=12, num_layers=params['layers'], hidden_units=params['neurons'])
    maml_train(model, train_tasks, params['inner_lr'], params['inner_steps'], params['outer_lr'], epochs=10)
    sharpe = evaluate(model, val_tasks)
    return sharpe

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)

C:\Users\User\AppData\Roaming\Python\Python312\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[I 2025-02-11 13:07:21,511] A new study created in memory with name: no-name-6ff007ee-e3a2-4df7-b834-fef408445eb8
C:\Users\User\AppData\Local\Temp\ipykernel_16156\1450247049.py:5: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'inner_lr': trial.suggest_loguniform('inner_lr', 1e-5, 1e-2),
C:\Users\User\AppData\Local\Temp\ipykernel_16156\1450247049.py:6: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'outer_lr': 

NameError: name 'train_tasks' is not defined

In [ ]:
def backtest(model, test_tasks):
    results = []
    for task in test_tasks:
        # Adapt model to task
        adapted_model = adapt_model(model, task['support'])
        
        # Generate signals
        signals = generate_signals(adapted_model, task['data'])
        
        # Calculate returns
        returns = calculate_returns(signals, task['data']['close'])
        
        # Compute metrics
        results.append({
            'sharpe': sharpe_ratio(returns),
            'sortino': sortino_ratio(returns),
            'max_drawdown': max_drawdown(returns),
            'profit_factor': profit_factor(returns)
        })
    return results

In [ ]:
def sharpe_ratio(returns, risk_free_rate=0):
    excess_returns = returns - risk_free_rate
    return excess_returns.mean() / excess_returns.std()

def sortino_ratio(returns, risk_free_rate=0):
    downside_returns = returns[returns < 0]
    return (returns.mean() - risk_free_rate) / downside_returns.std()

def max_drawdown(returns):
    cumulative = (1 + returns).cumprod()
    peak = cumulative.expanding(min_periods=1).max()
    return (cumulative / peak - 1).min()

def profit_factor(returns):
    gains = returns[returns > 0].sum()
    losses = -returns[returns < 0].sum()
    return gains / losses if losses != 0 else float('inf')